In [1]:
import functools

In [2]:
#Composing] a list function
compose = lambda *functions: functools.reduce(lambda f, g: lambda x: f(g(x)), functions, lambda x: x)

In [3]:
#some functions

def square(x):return x**2
def ceil(x):return round(x+.49)
def do_half(x):return x//2

In [4]:
compose(square,ceil,do_half)(7)  #calls function from right to left

9

In [5]:
#A more simplistic way
from typing import List

def composed_function(outputs, key, functions:List[callable]):
    for i,f in enumerate(functions[::-1]):
        f_n = f.__name__
        outputs[f_n] = f(outputs[key])
        key = f_n
    return outputs

composed_function({'x_in': 7}, 'x_in', [square,ceil,do_half])

{'x_in': 7, 'do_half': 3, 'ceil': 3, 'square': 9}

In [6]:
#lets use a decorator to introcuce the function first
def introduce_me_first(function:callable):
    def with_introduction(*args, **kwargs):
        print(f'Hello from {function.__name__}')
        output = function(*args, **kwargs)
        print(output)
        return output
    return with_introduction

In [7]:
@introduce_me_first
def square(x):return x**2

@introduce_me_first
def ceil(x):return round(x+.5)

@introduce_me_first
def do_half(x):return x//2

In [8]:
compose(square,ceil,do_half)(7) 

Hello from do_half
3
Hello from ceil
3
Hello from square
9


9

In [9]:
# deley your task by a second, useful for api testing, data logging 

import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def Print():
    print('I am as lazy as a Tortoise')
 
start = time.time()   
Print()
time.time()-start

I am as lazy as a Tortoise


1.0014557838439941

In [10]:
# useful for debugging 
import functools

DEBUG_FUNCTION = {'test':False} #if you want to debug the function

def debug(function):
    """Print the function signature and return output"""
    
    @functools.wraps(function)
    def wrapper_debug(*args, **kwargs):
        
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        
        print(f"Function: {function.__name__}, Signature: ({signature})")
        
        output = function(*args, **kwargs)
        
        print(f"{function.__name__!r} output: {output!r}")           # 4
        
        return output
    
    return wrapper_debug if DEBUG_FUNCTION.get(function.__name__, False) else function

In [11]:
@debug
def test(a:int,b:int)->int:
    return a+b

t = test(3,9)

In [12]:
# or you can do this....which is less flexible but it works
import functools
DEBUG_MODE = True

def debug(function, active = DEBUG_MODE):
    """Print the function signature and return output"""
    
    @functools.wraps(function)
    def wrapper_debug(*args, **kwargs):
        
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        
        print(f"Function: {function.__name__}, Signature: ({signature})")
        
        output = function(*args, **kwargs)
        
        print(f"{function.__name__!r} output: {output!r}")           # 4
        
        return output
    
    return wrapper_debug if active else function


@debug
def test(a:int,b:int)->int:
    return a+b

t = test(3,9)

Function: test, Signature: (3, 9)
'test' output: 12


In [13]:
# or you can do this 
from functools import partial
do_debugg = partial(debug, active = True)

@do_debugg
def test(a:int,b:int)->int:
    return a+b

t = test(3,9)

Function: test, Signature: (3, 9)
'test' output: 12


In [14]:
donot_debug = partial(debug, active = False)
@donot_debug
def test(a:int,b:int)->int:
    return a+b

t = test(3,9)

In [15]:
#Print the runtime of the decorated function

import functools
import time

def timer(function):
    """Print the runtime of the decorated function"""
    @functools.wraps(function)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()   
         
        value = function(*args, **kwargs)
        
        end_time = time.perf_counter()      
        
        run_time = end_time - start_time 
        print(f"Finished {function.__name__!r} in {run_time:.4f} secs")
        return value

In [16]:
# count how many times a function  is called

import functools

def count_calls(function):
    @functools.wraps(function)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"{function.__name__!r} function call: {wrapper_count_calls.num_calls}")
        return function(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls


@count_calls
def test(a:int,b:int)->int:
    return a+b

t = test(3,9)
t = test(3,9)
t = test(3,9)

'test' function call: 1
'test' function call: 2
'test' function call: 3


In [17]:
#lets make a class singleton!
import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

In [18]:
# let's do type conversion at runtime

def convert_dtype(*types):
    def converter(function):
        @functools.wraps(function)
        def _function(*args, **kwargs):
            converted_args = [type(arg)  for type,arg in zip(types, args)]
            
            print(converted_args)
            return function(*converted_args, **kwargs)
        return _function
    return converter
            
            
            
@convert_dtype(int, int)
def test(a:int,b:int)->int:
    return a+b

test(3, '9')

[3, 9]


12

In [21]:
import typeguard

@typeguard.typechecked
def test(a:int,b:int)->int:
    return a+b

test(3, '9')  # throws type error

TypeError: type of argument "b" must be int; got str instead

In [24]:
@convert_dtype(int, int)   
@typeguard.typechecked
def test(a:int,b:int)->int:
    return a+b

test(3, '9')  # works perfectly, our type converter convert type beforehand

[3, 9]


12